In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ### loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Jayanth and I am an AI Engineer")])

AIMessage(content="Hi Jayanth,\n\nIt's nice to meet you! I'm glad to know you're an AI Engineer. That's a fascinating field. \n\nWhat kind of AI work do you do?  Are you working on anything interesting at the moment?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 22, 'total_tokens': 80, 'completion_time': 0.105454545, 'prompt_time': 0.000377219, 'queue_time': 0.019487730999999998, 'total_time': 0.105831764}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f1a23de-18da-47c9-a892-4aa0254ad618-0', usage_metadata={'input_tokens': 22, 'output_tokens': 58, 'total_tokens': 80})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Jayanth and I am a  AI Engineer"),
        AIMessage(content="Hello Jayanth, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or feelings, but I'm always eager to learn and help. \n\nWhat can I do for you today?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Jayanth and that you are an AI Engineer! \n\nIs there anything else you'd like to tell me about yourself or would you like to talk about something else? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 93, 'total_tokens': 139, 'completion_time': 0.083636364, 'prompt_time': 0.005059398, 'queue_time': 0.019650851, 'total_time': 0.088695762}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fde8d3a2-f078-4ce2-8507-b97f71335230-0', usage_metadata={'input_tokens': 93, 'output_tokens': 46, 'total_tokens': 139})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Jayanth and I am a AI Engineer")],
    config=config
)

In [8]:
response.content

"Hello Jayanth! It's nice to meet you. \n\nAs an AI myself, I'm always interested in connecting with other people working in this field. What kind of AI engineering work are you involved in? \n\nI'm curious to hear about your projects and experiences.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Jayanth.  I remember that from our first conversation! 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 98, 'total_tokens': 118, 'completion_time': 0.036363636, 'prompt_time': 0.00247608, 'queue_time': 0.02020036, 'total_time': 0.038839716}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e663e046-c564-4da7-91c1-12b148ca8390-0', usage_metadata={'input_tokens': 98, 'output_tokens': 20, 'total_tokens': 118})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Naman")],
    config=config1
)
response.content

"Hi Naman, it's nice to meet you! 👋\n\nIs there anything I can help you with today?\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Naman,  remember? 😄 \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish! It's nice to meet you. \n\nHow can I help you today?  😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 30, 'total_tokens': 57, 'completion_time': 0.049090909, 'prompt_time': 0.000143589, 'queue_time': 0.020947751, 'total_time': 0.049234498}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-334866fa-3d42-47de-b615-b18d3e0e0b21-0', usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Jayanth")],
    config=config
)

response

AIMessage(content="Hi Jayanth, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  What can I help you with today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 31, 'total_tokens': 73, 'completion_time': 0.076363636, 'prompt_time': 0.001840812, 'queue_time': 0.019820795, 'total_time': 0.078204448}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5f2a6042-3850-4b36-aba6-4b0dbd5585e1-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jayanth.  \n\nI remember that from our introduction!  Is there anything else I can help you with?\n'

In [18]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Jayanth")],"language":"Telugu"})
response.content

'మీకు కల్యాణ బాధ్యతలు, జయంత్! 😊 \n\nఏంటి, మీరు ఏదైనా అడిగి చూడాలనుకుంటున్నారా?  \n నేను మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాను. \n'

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Jayanth")],"language":"Telugu"},
    config=config
)
repsonse.content

'Hello Jayanth, \n\nనేను మీకు సహాయపడగల ఎల్లవే సహాయకుడిని. మీరు ఏమి అడిగితే అందరికీ నేను జవాబు ఇస్తాను. 😊\n\nమీకు ఏమి సాయం చేయాలి? \n\n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)

In [23]:
response.content


'మీ పేరు జయంత్. 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [30]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=80,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You said you like vanilla ice cream!  🍦 😊'

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2" 😊  \n\n\n'

In [33]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob!  😊 \n'

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "whats 2 + 2" \n\n\n\nLet me know if you\'d like to try another one!\n'